In [107]:
import pandas as pd
import os
from nltk.corpus import stopwords
import nltk.data
import logging
import numpy as np  # Make sure that numpy is imported
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier 
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [108]:
import warnings
warnings.filterwarnings('ignore')

In [109]:
#Loading Test Datasets 
train = pd.read_csv("./data/labeledTrainData.tsv",header=0,delimiter="\t",quoting=3)
unlabeled_train = pd.read_csv("./data/unlabeledTrainData.tsv",header=0,delimiter="\t",quoting=3)
test = pd.read_csv("./data/testData.tsv",header=0,delimiter="\t",quoting=3)

In [110]:
#checking size of Dataset  
print(len(train["review"])+len(unlabeled_train["review"])+len(test["review"]))

100000


100,000 total reviews out of which 25,000 is test set

In [111]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review,"lxml").get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words) 

In [7]:


# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences 

In [112]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)



Parsing sentences from training set
Parsing sentences from unlabeled set


In [113]:
print(len(sentences))

795538


In [114]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name) 

Training model...


In [115]:
model.most_similar("man")

[('woman', 0.612058162689209),
 ('lady', 0.5854551196098328),
 ('lad', 0.5561214685440063),
 ('monk', 0.5500044822692871),
 ('boy', 0.525436520576477),
 ('guy', 0.5210829973220825),
 ('chap', 0.5168739557266235),
 ('farmer', 0.5157365202903748),
 ('men', 0.5148264169692993),
 ('soldier', 0.5121333599090576)]

In [116]:
model.most_similar("queen")

[('princess', 0.6477822065353394),
 ('bride', 0.6134942173957825),
 ('victoria', 0.5962779521942139),
 ('showgirl', 0.5875634551048279),
 ('belle', 0.5832908153533936),
 ('stepmother', 0.5819955468177795),
 ('eva', 0.577735424041748),
 ('maid', 0.5742698907852173),
 ('regina', 0.5687487125396729),
 ('kristel', 0.5616047382354736)]

In [117]:
model.doesnt_match("man woman child kitchen".split())


'kitchen'

In [118]:
model.doesnt_match("france england germany berlin".split())

'berlin'

In [119]:
model.doesnt_match("Zoya berlin london austria".split())

'austria'

In [120]:
model.most_similar("awful")

[('terrible', 0.7705107927322388),
 ('horrible', 0.7305666208267212),
 ('atrocious', 0.7290143966674805),
 ('abysmal', 0.7157675623893738),
 ('dreadful', 0.7105789184570312),
 ('appalling', 0.6870583891868591),
 ('horrendous', 0.6789425015449524),
 ('horrid', 0.6385958194732666),
 ('amateurish', 0.6287796497344971),
 ('lousy', 0.6039535403251648)]

In [121]:
model.syn0.shape

(16490, 300)

The number of rows in syn0 is the number of words in the model's vocabularl, and the number of columns corresponds to the size of the feature vectore, which we set in Part2. Setting the minimum word count to 40 gave us a total vocabularly of 16,492 words with 300 feature apiece. 

In [122]:
model["man"]

array([ 0.01139142,  0.03010188,  0.05542311, -0.04067303,  0.0245908 ,
       -0.01058377,  0.12768008, -0.01556714, -0.03268285, -0.07448141,
        0.04596307,  0.04924469, -0.00575278, -0.00780511,  0.01648034,
       -0.05519637,  0.04220185, -0.04801417,  0.03341379, -0.04681101,
        0.0461699 , -0.06256   ,  0.05420063, -0.04261733,  0.00258828,
        0.02086867,  0.00452781,  0.06171165, -0.05630255,  0.0148612 ,
       -0.04682961,  0.09298653, -0.01528321,  0.05393535, -0.10343533,
        0.00228477,  0.03131061,  0.07139163, -0.00125286, -0.09141771,
        0.04023457,  0.00287645, -0.04564029,  0.05561365, -0.02993594,
       -0.00360389, -0.02187949,  0.04369891,  0.01412906,  0.05241794,
       -0.09993239,  0.03676849,  0.07900926,  0.07839028, -0.03541839,
        0.00170283, -0.22913581,  0.09532517,  0.10355403, -0.06231165,
       -0.00648718,  0.0689905 ,  0.04212997, -0.0203238 ,  0.0009045 ,
        0.01728283, -0.0914805 , -0.13602796,  0.02202323,  0.02

### From Words to Paragraphs,Attempt 1: Vector Averaging 

One challenge with the IMDB dataset is the variable-lenght reviews. We need to find a way to take individual word vectors and transform them into a **feature set that is the same length for every review**  

Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each review. **One method was to simply average the word vectors in a given review**  

In [123]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each review a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
        #
        # Print a status message every 1000th review
        if counter%1000. == 0.:
            print("Review %d of %d" % (counter, len(reviews)))

        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
               num_features)

        # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [124]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [125]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...") 
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "rf_Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


![](./data/rf_word2vec_average_vector.png)

In [126]:
# Fit a Logistic Regression to the training data, using 100 trees
from sklearn.linear_model import LogisticRegression 
lr = LogisticRegression() 

print("Fitting a Logsitic Regression model to labeled training data...") 
lr = lr.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = lr.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "lr_Word2Vec_AverageVector.csv", index=False, quoting=3 )

Fitting a Logsitic Regression model to labeled training data...


![](./data/lr_word2Vec_averagee_vector.png)

This approach produced results much better than chance , but underperformed Bag of Words by a few percentage points.  

Since the elements-wise average of the vectors didn't produce spectacular results, perhaps we could do it in a more intelligent way. 
A standard way of weighting word vectors is to apply "tf-idf" weights,which measures how important a given word is within a given set of documents. One way to extract tf-idf weights in python is by using scikit-learn's TfidfVectorizer,. When we tried weighting our word vectors in this way,we found no substantial improvement in performance.

### From Words to Paragraphs, Attempt 2: Clustering 

Word2Vec creates clusters of semantically related words,so another possible approach is to exploit the similarity of words within a cluster. Grouping vectors in this way is known as "Vector Quantization". To accomplish this,we first need to find the centers of the word clusters,which we can do by using a clustering algorithm such as K-means

In K_Means , the one parameter we need to set is "K",or the number of clusters.How should we decide how many clusters to create? Trial and error suggested that small clusters,with an average of only 5 words or so per cluster ,gave better results than large clusters with many words. 

In [127]:
from sklearn.cluster import KMeans 
import time 

start = time.time() 
# Set "K" (num_clusters) to be 1/5th of the Vocabulary size, or an average of 5 words per cluster
word_vectors = model.syn0 
num_clusters = int(word_vectors.shape[0]/5)
print(num_clusters)
# Initialize a K-means object and use it to extract centroids 
kmeans_clustering = KMeans(n_clusters = num_clusters) 
idx = kmeans_clustering.fit_predict(word_vectors)  

#Get the end time and print how long the process took 
end = time.time()
elapsed = end - start  
print("Time taken for K Means clustering: ",elapsed,"seconds")

3298
Time taken for K Means clustering:  738.3951489925385 seconds


The cluster assignment for each word is now stored in idx,and the vocabulary from our original Word2Vec model is still stored in model.index2word . For convenience we ,zip these into one dictionary as follows:

In [83]:
#create a Word / index dictionary , maooing each vocabulary word to a cluster number 

word_centroid_map = dict(zip(model.index2word,idx))

This is a little abstract ,so let's take a closer look at what our clusters contain. Your clusters may differ, as Word2Vec relies on a random number seed. Here is a loop that prints out the words for cluster 0 through 9 

In [128]:
# Top 20 clusters
for cluster in range(0,20):
    print("\nCluster %d" % cluster) 
    words = [] #Finding words for that cluster 
    for key,value in word_centroid_map.items():
        if(value==cluster):
            words.append(key) 
    print(words)


Cluster 0
['training']

Cluster 1
['jolly', 'elf', 'legged', 'merry']

Cluster 2
['degradation', 'unspeakable', 'perpetual']

Cluster 3
['biographical', 'somerset', 'sleuth', 'maugham', 'excalibur', 'adapted', 'originated', 'deathtrap', 'steinbeck', 'hellman', 'masterwork', 'prose', 'macbeth', 'hammett', 'novella']

Cluster 4
['locks', 'holding', 'burying', 'locking']

Cluster 5
['pleasence', 'hare', 'donald', 'pleasance', 'sutherland']

Cluster 6
['motifs', 'cues']

Cluster 7
['pauline', 'davidson', 'sadie']

Cluster 8
['overplayed', 'emotionless', 'overacted', 'unemotional']

Cluster 9
['raven', 'sheep']

Cluster 10
['precise', 'sparse', 'noticeable', 'rapid', 'muted', 'sporadic', 'jarring']

Cluster 11
['closet', 'beds', 'pool', 'attic', 'room', 'towel', 'backyard', 'bed', 'bedroom', 'flowers', 'bathroom', 'garage', 'classroom', 'mirror', 'kitchen', 'front', 'basement']

Cluster 12
['cohen', 'michael', 'sam', 'andrew', 'matheson', 'neil', 'labute', 'reiner']

Cluster 13
['tilly', '

We can see that clusters are of varying quality. Most of them make sense

At any rate,now we have a cluster (or "centroid") assignment for each word , and we can define a function to convert reviews into bags-of-centroids. This works just like Bag of Words but uses semantically related clusters instead of individual words. 

In [129]:
def create_bag_of_centroids(wordlist,word_centroid_map):
    #THe number of clusters is equal to the highest cluster index in word_centroid_map 
    num_centroids = max(word_centroid_map.values()) + 1 
    
    #Pre-allocate the bag of centroid vector (for speed) 
    bag_of_centroids = np.zeros(num_centroids,dtype = "float32") 
    
    #Loop pver the the words in the review. If the word is in the vocabularly.
    # Find which cluster it belongs to, and increment that cluster count by one. 
    
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word] 
            bag_of_centroids[index] +=1 
    return bag_of_centroids
            

The Function above will give us a numy array for each review,each with a number of features equal to the number of clusters. Finally we create bags of centroid for our training and test set, then train a random forest and extract results

In [130]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1


In [131]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier()

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "rf_bagofcentroids.csv", index=False, quoting=3 ) 

Fitting a random forest to labeled training data...


![](./data/rf_bagof_centroids.png)

In [133]:
# Fit a random forest and extract predictions 
lr = LogisticRegression()

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
lr = lr.fit(train_centroids,train["sentiment"])
result = lr.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "lr_bagofcentroids.csv", index=False, quoting=3 ) 

Fitting a random forest to labeled training data...


![](./data/lr_bagof_centroids.png) 

In [ ]:
#GBM
from sklearn.ensemble import GradientBoostingClassifier

gbm =GradientBoostingClassifier()
gbm=gbm.fit(train_centroids,train["sentiment"])

result= gbm.predict(test_centroids) 
output1 = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output1.to_csv( "gbm_bagofcentroids.csv", index=False, quoting=3)  

In [106]:
#XGB
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
xgb =  XGBClassifier()
xgb.fit(train_centroids,train["sentiment"])
result1 = xgb.predict(test_centroids) 
output1 = pd.DataFrame( data={"id":test["id"], "sentiment":result1} )
output1.to_csv( "xgb_bagofcentroids.csv", index=False, quoting=3) 

In [135]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
